In [2]:
import os
import json
from src.data import CustomDataset, OriginalDataset
from transformers import AutoTokenizer

"""
id | 원본 데이터 | 전처리 데이터 | inference | label | total score (mean) | rouge1 | BERTScore | BLEURT
-> mean 기준 오름차순 정렬 csv
"""
# dev data result 넣기
JSON_PATH = "results/dev/dev_result_preprocess_yi.json"
DEV_DATA_PATH = "resource/data/일상대화요약_dev.json"
MODEL_ID = "hyeogi/Yi-6b-dpo-v0.2"
DETAIL = "no_ngram"

with open(JSON_PATH, "r") as f:
    result_data = json.load(f)

## utils Config File :  config_yi.json


In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
dataset = OriginalDataset(DEV_DATA_PATH, tokenizer)
dataset_preprocessed = CustomDataset(DEV_DATA_PATH, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ens

In [4]:
tokenizer.decode(dataset[0], skip_special_tokens=True)

' [INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]과 [대화 키워드]를 보고, 한국어 대화 요약문을 생성해주세요.\n        \n<</SYS>>\n\n[대화 키워드] : 식품, 건강, 검사에 대한 대화 내용입니다.\n[대화 내용]\nSD2000691: 근데 인제 건강이 우리만 막 건강한다고 되는 게 아니잖아. 인제 애들도 건강도 챙겨줘야 되고 그러잖아. 그러니까 너는 애들한테 뭐 따로 먹이는 뭐 식품 뭐 이런 거 있어?\nSD2000692: 유산 유산균하고 비타민 그런데 애들이 잘 안 먹지. 나도 잘 안 먹는 데 애들이 먹나? 근데 쫓아다니면서 챙겨줄 수도 없고 그런데 유산균은 꼭 먹이라 그러더라고. 유산균 장이 건강해야지 전체적으로 다 좋아진다고 그러더라고.\nSD2000692: 그래서 될 수 있으면 비타민도 사놓기는 했는데 그거는 넘어가더라도 나는 유산균은 꼭 먹으라고 얘기한다. 유산균이 어떤 사람은 공복에 먹으라 그러고 어떤 사람은 그냥 아무 때나 먹으라고 하는데 모르겠어. 언제 먹는 게 좋은지\nSD2000692: 그래서 name2이는 또 천식에 아토피까지 있잖아. 진짜 우리 name2이는 건강을 누구보다 진짜 걔는 정말 신경 써가지고 지켜야 되는데 지금은 애들 유산균 먹이는 거에 집중하지 유산균이 젤 좋다고는 하더라고. 그 뭐 피부나 아니면 장내 활동이 좋아야지 감기나 뭐 이런 것도 걸렸을 때 빨리 낫는다고.\nSD2000692: 그래서 근데 어떤 유산균이 또 좋은지 몰라. 가루로 된 걸 먹이는 사람도 있고 알약으로 된 걸 먹이는 사람도 있는데 나는 그냥 가루로 된 거 그리고 광고 많이 나오는 거 어~ 나도 거기에 대한 정보가 많이 없어서 그냥 오래된 회사 거? 이런 데 거 먹이고\nSD2000692: 그래서 애들이 건강해야 나도 편하기는 한데 그~ 애들 건강까지 챙겨준다는 것도 어렵고 음~ 지금은 유산균만 먹여. 으 유산균 먹이고 야채 많이 먹이고 그러는데 야채도 잎채소랑 줄기랑 또 다르대. 영양분이 그러니까 

In [5]:
tokenizer.decode(dataset_preprocessed[0], skip_special_tokens=True)

' [INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]과 [대화 키워드]를 보고, 한국어 대화 요약문을 생성해주세요.\n        \n<</SYS>>\n\n[대화 키워드] : 식품, 건강, 검사에 대한 대화 내용입니다.\n[대화 내용]\nSD2000691: 근데 건강이 우리만 건강한다고 되는 게 아니잖아. 애들도 건강도 챙겨줘야 되고 그러잖아. 그러니까 너는 애들한테 따로 먹이는 식품 이런 거 있어?\nSD2000692: 유산 유산균하고 비타민 그런데 애들이 잘 안 먹지. 나도 잘 안 먹는 데 애들이 먹나? 근데 쫓아다니면서 챙겨줄 수도 없고 그런데 유산균은 꼭 먹이라 그러더라고. 유산균 장이 건강해야지 전체적으로 다 좋아진다고 그러더라고.\nSD2000692: 그래서 될 수 있으면 비타민도 사놓기는 했는데 그거는 넘어가더라도 나는 유산균은 꼭 먹으라고 얘기한다. 유산균이 어떤 사람은 공복에 먹으라 그러고 어떤 사람은 그냥 아무 때나 먹으라고 하는데 모르겠어. 언제 먹는 게 좋은지\nSD2000692: 그래서 또 천식에 아토피까지 있잖아. 진짜 우리 건강을 누구보다 진짜 걔는 정말 신경 써가지고 지켜야 되는데 지금은 애들 유산균 먹이는 거에 집중하지 유산균이 젤 좋다고는 하더라고. 피부나 아니면 장내 활동이 좋아야지 감기나 이런 것도 걸렸을 때 빨리 낫는다고.\nSD2000692: 그래서 근데 어떤 유산균이 또 좋은지 몰라. 가루로 된 걸 먹이는 사람도 있고 알약으로 된 걸 먹이는 사람도 있는데 나는 그냥 가루로 된 거 그리고 광고 많이 나오는 거 나도 거기에 대한 정보가 많이 없어서 그냥 오래된 회사 거? 이런 데 거 먹이고\nSD2000692: 그래서 애들이 건강해야 나도 편하기는 한데 애들 건강까지 챙겨준다는 것도 어렵고 지금은 유산균만 먹여. 유산균 먹이고 야채 많이 먹이고 그러는데 야채도 잎채소랑 줄기랑 또 다르대. 영양분이 그러니까 뭐를 먹여야지 건강에 더 좋은지 어렵지 다 찾아서 주기가\nSD2000692: 너는 

In [6]:
import evaluate
rouge = evaluate.load('rouge')
bert_score = evaluate.load('bertscore')
bleurt = evaluate.load('bleurt', 'bleurt-large-512', module_type="metric")


def compute_metrics(label, pred):
    # Simple postprocessing
    pred, label = postprocess_text(pred, label)

    rouge_scores = rouge.compute(predictions=[pred], references=[label], rouge_types=["rouge1"])
    # rouge_scores = rouge.get_scores(predictions, labels, avg=True)
    bertScore = bert_score.compute(predictions=[pred], references=[label], lang="ko")['f1'][0]
    bleurtScore = bleurt.compute(predictions=[pred], references=[label])['scores'][0]

    rouge1 = rouge_scores['rouge1']

    # print(bertScore, bleurtScore, rouge1)
    total = (bertScore + bleurtScore + rouge1) / 3

    return {"total" : round(total, 4), "rouge1" : round(rouge1, 4), "BERTScore" : round(bertScore, 4), "BLEURT": round(bleurtScore, 4)}

def postprocess_text(pred, label):
    return pred.strip(), label.strip()

In [7]:
result_data[0]['inference']

'1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 언제인가요? 3. 환불이 불가능하다면, 환불이 가능한 시점은 언제인가요?'

In [8]:
result_data[0]['output']

'두 화자는 이 대화에서 병원 진료와 자녀를 위한 건강식품에 대해 말했습니다. SD2000691은 자녀들에게 유산균과 종합 비타민을 챙겨준다고 말했습니다. 그런데 고등학교 수험생인 딸아이 건강을 위해 홍삼 제품을 먹였더니 생리통이 심해지고 혹이 생겼다고 말했고, 병원에서 홍삼 제품에 여성호르몬이 많으니 복용하지 말라고 했다며 속상했다고 했습니다. 한편 한동안 부종이 생겼었다며 상태가 심해지면 병원 진료를 받을 계획이라고 말했습니다. SD2000692는 자녀들에게 건강에 좋은 채소와 유산균, 비타민 등의 영양제를 준다고 했습니다. 그리고 폐렴 접종은 금액도 비싸고 효능에 대한 확신이 없어서 고민했었는데 조만간 접종을 받는 걸로 마음을 바꿨다고 했습니다.'

In [9]:
compute_metrics(result_data[0]['inference'], result_data[0]['output'])

{'total': 0.2219, 'rouge1': 0.0, 'BERTScore': 0.5799, 'BLEURT': 0.0858}

In [10]:
import pandas as pd
import tqdm

result_df = pd.DataFrame(columns=['id', 'original', 'preprocessed', 'inference', 'label', 'total', 'rouge1', 'BERTScore', 'BLEURT'])

for idx in tqdm.tqdm(range(len(result_data))):
    metrics = compute_metrics(result_data[idx]['inference'], result_data[idx]['output'])
    row = [result_data[idx]['id'],
           tokenizer.decode(dataset[idx], skip_special_tokens=True), 
           tokenizer.decode(dataset_preprocessed[idx], skip_special_tokens=True),
           result_data[idx]['inference'],
           result_data[idx]['output'],
           metrics['total'],
           metrics['rouge1'],
           metrics['BERTScore'],
           metrics['BLEURT']]
    result_df.loc[len(result_df)] = row

100%|██████████| 102/102 [00:15<00:00,  6.66it/s]


In [11]:
result_df

,id,original,preprocessed,inference,label,total,rouge1,BERTScore,BLEURT
0,nikluge-2024-일상 대화 요약-dev-000001,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",두 화자는 이 대화에서 병원 진료와 자녀를 위한 건강식품에 대해 말했습니다. SD2...,0.2219,0.0,0.5799,0.0858
1,nikluge-2024-일상 대화 요약-dev-000002,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",두 화자는 이 대화에서 좋아하거나 싫어하는 음식에 대해 말했습니다. SD200069...,0.1996,0.0,0.5642,0.0345
2,nikluge-2024-일상 대화 요약-dev-000003,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 100% 환불이 안된다면, 환불이 가능한 시...",두 화자는 이 대화에서 음식을 먹는 방법에 대해 말했습니다. SD2000695는 최...,0.2141,0.0,0.5802,0.0620
3,nikluge-2024-일상 대화 요약-dev-000004,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",두 사람은 이 대화에서 성격에 관해 이야기 나눴습니다. SD2000696은 엉뚱하고...,0.1885,0.0,0.5632,0.0022
4,nikluge-2024-일상 대화 요약-dev-000005,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",두 사람은 이 대화에서 엠비티아이와 혈액형별 성격 등에 관해 이야기 나눴습니다. S...,0.2068,0.0,0.5762,0.0444
...,...,...,...,...,...,...,...,...,...
97,nikluge-2024-일상 대화 요약-dev-000098,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",두 화자는 이 대화에서 선물에 대해 말했습니다. SD2000867은 물질적이진 않지...,0.1964,0.0,0.5792,0.0101
98,nikluge-2024-일상 대화 요약-dev-000099,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",두 화자는 이 대화에서 선물에 대해 말했습니다. SD2000867은 아빠에게 받은 ...,0.1879,0.0,0.5717,-0.0080
99,nikluge-2024-일상 대화 요약-dev-000100,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...","SD2000867과 SD2000868은 건강 관리를 위한 운동, 식단, 보조제에 대...",0.1906,0.0,0.5794,-0.0077
100,nikluge-2024-일상 대화 요약-dev-000101,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...","SD2000867과 SD2000868은 다이어트, 건강에 대해 대화를 나눴습니다. ...",0.1818,0.0,0.5792,-0.0339


In [12]:
result_df.sort_values(by='total')

,id,original,preprocessed,inference,label,total,rouge1,BERTScore,BLEURT
13,nikluge-2024-일상 대화 요약-dev-000014,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",이 대화에서 두 화자는 첫 아르바이트에 대해 이야기를 나눴습니다. SD2000724...,0.1647,0.0000,0.5519,-0.0578
27,nikluge-2024-일상 대화 요약-dev-000028,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",이 대화에서 화자들은 선호하는 계절과 싫어하는 계절에 대해서 이야기했습니다. SD2...,0.1688,0.0000,0.5630,-0.0565
8,nikluge-2024-일상 대화 요약-dev-000009,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...","두 화자는 이 대화에서 피서, 날씨에 관한 대화를 나눴습니다. SD2000909는 ...",0.1700,0.0000,0.5582,-0.0482
50,nikluge-2024-일상 대화 요약-dev-000051,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",이 대화에서 화자들은 추억의 음식에 대해 이야기했습니다. SD2000789는 예전 ...,0.1732,0.0000,0.5709,-0.0511
47,nikluge-2024-일상 대화 요약-dev-000048,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시기는 ...",두 화자는 이 대화에서 학교 친구의 코골이로 인한 피해와 해결 방법에 대해 말했습니...,0.1746,0.0000,0.5526,-0.0287
...,...,...,...,...,...,...,...,...,...
6,nikluge-2024-일상 대화 요약-dev-000007,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 100% 환불이 안된다면, 환불이 가능한 시...",두 화자는 이 대화에서 지역 먹거리와 식당에 대해 말했습니다. SD2000703은 ...,0.3094,0.2500,0.5934,0.0848
94,nikluge-2024-일상 대화 요약-dev-000095,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",두 화자는 이 대화에서 꿈에 대해 말했습니다. SD2000863은 다이어리 꾸미기가...,0.3290,0.4000,0.5483,0.0386
95,nikluge-2024-일상 대화 요약-dev-000096,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...","SD2000865와 SD2000866은 고등학교 시절 동아리, 선생님, 친구에 대해...",0.3470,0.5000,0.5439,-0.0028
34,nikluge-2024-일상 대화 요약-dev-000035,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,[INST] <<SYS>>\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]...,"1. 100% 환불이 가능한가요? 2. 환불이 불가능하다면, 환불이 가능한 시점은 ...",두 화자는 이 대화에서 기억과 추억에 남는 여행지에 대해 말했습니다. SD20007...,0.3476,0.4444,0.5574,0.0409


In [13]:
from datetime import datetime
now = datetime.now()
result_df.to_csv(os.path.join("results/dev/", f"{MODEL_ID.split('/')[1]}_{DETAIL}_preprocessed_time_{now.strftime('%Y-%m-%d_%H:%M')}.csv"), index=False)